Data set: 16*16 grayscale 10 classes
- digits.mat
- X 5000*256
- y 5000*1
- Xvalid 5000*256
- yvalid 5000*1
- Xtest 1000*256
- ytest 1000*1

In [44]:
import numpy as np
from scipy.io import loadmat

In [45]:
def standardizeCols(M, mu=None, sigma2=None):
    """
    Standardize each column of matrix M to have zero mean and unit standard deviation.
    
    Parameters:
    M (numpy.ndarray): The input matrix.
    mu (numpy.ndarray, optional): Precomputed mean of the columns.
    sigma2 (numpy.ndarray, optional): Precomputed standard deviation of the columns.
    
    Returns:
    numpy.ndarray: The standardized matrix.
    numpy.ndarray: Mean of the columns.
    numpy.ndarray: Standard deviation of the columns.
    """
    M = M.astype(float)  # Ensure M is float for precision
    nrows, ncols = M.shape

    if mu is None or sigma2 is None:
        mu = np.mean(M, axis=0)
        sigma2 = np.std(M, axis=0)
        sigma2[sigma2 < np.finfo(float).eps] = 1  # Avoid division by zero

    S = M - mu  # Subtract mean
    if ncols > 0:
        S = S / sigma2  # Divide by standard deviation

    return S, mu, sigma2

In [47]:
# Forward pass
def MLPclassificationPredict(w, X, nHidden, nLabels):
    # w: 1-D, stores all weights
    nInstances, nVars = X.shape

    # Form Weights
    offset = 0
    inputWeights = w[offset:nVars*nHidden[0]].reshape(nVars, nHidden[0])
    offset += nVars*nHidden[0]
    hiddenWeights = []
    for h in range(1, len(nHidden)):
        size = nHidden[h-1] * nHidden[h]
        hiddenWeights.append(
            w[offset:offset+size].reshape(nHidden[h-1], nHidden[h]))
        offset += size
    outputWeights = w[offset:offset+nHidden[-1]
                      * nLabels].reshape(nHidden[-1], nLabels)

    # Compute Output
    y = np.zeros((nInstances, nLabels))
    for i in range(nInstances):
        ip = [X[i] @ inputWeights]
        fp = [np.tanh(ip[0])]
        for h in range(1, len(nHidden)):
            ip.append(fp[h-1] @ hiddenWeights[h-1])
            fp.append(np.tanh(ip[h]))
        y[i] = fp[-1] @ outputWeights

    # Pick the class with the highest score
    y = np.argmax(y, axis=1, keepdims=True) + 1
    return y

In [48]:
def sech2(x):
    return 1 - np.tanh(x)**2

# Loss and gradient
def MLPclassificationLoss(w, X, y, nHidden, nLabels):
    # X, y are all 2-D arrays
    nInstances, nVars = X.shape

    # Form Weights
    offset = 0
    inputWeights = w[offset:nVars*nHidden[0]].reshape(nVars, nHidden[0])
    offset += nVars*nHidden[0]
    hiddenWeights = []
    for h in range(1, len(nHidden)):
        size = nHidden[h-1] * nHidden[h]
        hiddenWeights.append(
            w[offset:offset+size].reshape(nHidden[h-1], nHidden[h]))
        offset += size
    outputWeights = w[offset:offset+nHidden[-1]
                      * nLabels].reshape(nHidden[-1], nLabels)

    f = 0  # Loss
    gInput = np.zeros_like(inputWeights)  # Gradient for input weights
    gHidden = [np.zeros_like(hw) for hw in hiddenWeights]  # Gradient for hidden weights
    gOutput = np.zeros_like(outputWeights)  # Gradient for output weights


    # Compute Output
    for i in range(nInstances):
        ip = [None] * len(nHidden)
        fp = [None] * len(nHidden)

        ip[0] = X[i, :] @ inputWeights  # 1-D array
        fp[0] = np.tanh(ip[0])  # 1-D array
        for h in range(1, len(nHidden)):
            ip[h] = fp[h-1] @ hiddenWeights[h-1]
            fp[h] = np.tanh(ip[h])
        yhat = fp[-1] @ outputWeights

        relativeErr = yhat - y[i, :]
        f += np.sum(relativeErr**2)

        err = 2 * relativeErr

        # Output Weights Gradient
        gOutput += np.outer(fp[-1], err)

        if len(nHidden) > 1:
            backprop = np.zeros_like(fp[-1])
            for c in range(nLabels):
                backprop[c] = err[c] * sech2(ip[-1]) * outputWeights[:, c].T
                gHidden[-1] += np.outer(fp[-2], backprop[c])
            backprop = np.sum(backprop, axis=0)  # Sum over classes

            # Other Hidden Layer Gradients
            for h in range(len(nHidden)-2, -1, -1):
                backprop = (backprop @ hiddenWeights[h].T) * sech2(ip[h])
                if h > 0:
                    gHidden[h-1] += np.outer(fp[h-1], backprop)
                else:
                    # Input weights
                    gInput += np.outer(X[i, :], backprop)
        else:
            # Input Weights Gradient for single hidden layer case
            for c in range(nLabels):
                gInput += err[c] * np.outer(X[i], sech2(ip[-1]) * outputWeights[:, c].T)

    # Put Gradient into vector
    g = np.zeros_like(w)
    g[:nVars * nHidden[0]] = gInput.reshape(-1, 1)
    offset = nVars * nHidden[0]
    for h in range(len(nHidden)-1):
        g[offset : offset + nHidden[h] * nHidden[h+1]] = gHidden[h].reshape(-1, 1)
        offset += nHidden[h] * nHidden[h+1]
    g[offset : offset + nHidden[-1] * nLabels] = gOutput.reshape(-1, 1)

    return f, g

In [49]:
data = loadmat('digits.mat')
X = data['X']
y = data['y'].flatten()
yvalid = data['yvalid']
ytest = data['ytest']

n, d = X.shape  # 5000, 256
nLabels = np.max(y)  # 10
yExpanded = 2 * np.eye(nLabels)[y - 1] - 1  # turn into one-hot vector
t = data['Xvalid'].shape[0]  # 5000
t2 = data['Xtest'].shape[0]  # 1000


# Standardize columns and add bias
X, mu, sigma = standardizeCols(X)
X = np.hstack([np.ones((n, 1)), X])

# Apply the same transformation to the validation/test data
Xvalid, _, _ = standardizeCols(data['Xvalid'], mu, sigma)
Xvalid = np.hstack([np.ones((t, 1)), Xvalid])

Xtest, _, _ = standardizeCols(data['Xtest'], mu, sigma)
Xtest = np.hstack([np.ones((t2, 1)), Xtest])


# Choose network structure
nHidden = [10]
d += 1  # Adjust (n, d) = X.shape

# Count number of parameters
nParams = d * nHidden[0]  # Input layer and first hidden layer
nParams += sum(nHidden[h-1] * nHidden[h] for h in range(1, len(nHidden)))
nParams += nHidden[-1] * nLabels  # Last hidden layer and output layer
# initialize weights
w = np.random.randn(nParams, 1)

# Train with stochastic gradient
maxIter = 100000
stepSize = 1e-3

for iter in range(0, maxIter):
    if (iter + 1) % (maxIter // 20) == 0:
        yhat = MLPclassificationPredict(w, Xvalid, nHidden, nLabels)
        validation_error = np.mean(yhat != yvalid)
        print(f'Training iteration = {iter + 1}, \
            validation error = {validation_error:.6f}')

    # batch = 1
    i = np.random.randint(n)
    f, g = MLPclassificationLoss(w, X[i:i+1], yExpanded[i:i+1], nHidden, nLabels)
    w -= stepSize * g

# Evaluate test error
yhat = MLPclassificationPredict(w, Xtest, nHidden, nLabels)
test_error = np.mean(yhat != ytest)
print(f'Test error with final model = {test_error:.6f}')

Training iteration = 5000,             validation error = 0.611800
Training iteration = 10000,             validation error = 0.608400
Training iteration = 15000,             validation error = 0.577800
Training iteration = 20000,             validation error = 0.569000
Training iteration = 25000,             validation error = 0.557800
Training iteration = 30000,             validation error = 0.520000
Training iteration = 35000,             validation error = 0.529600
Training iteration = 40000,             validation error = 0.496800
Training iteration = 45000,             validation error = 0.490400
Training iteration = 50000,             validation error = 0.523800
Training iteration = 55000,             validation error = 0.497600
Training iteration = 60000,             validation error = 0.476000
Training iteration = 65000,             validation error = 0.488200
Training iteration = 70000,             validation error = 0.491000
Training iteration = 75000,             validatio